In [3]:
import charts
import pymongo
from datetime import date,timedelta

Server running in the folder C:\Users\ZhangLi at 127.0.0.1:57268


In [4]:
client = pymongo.MongoClient('localhost',27017)
company_info = client['company_info']
information = company_info['information_1'] # 去数据库中information_1中的数据

In [53]:
for i in information.find(): # 改变月份的格式
    date_i = i['date'].split('-')
    if len(date_i) == 1:
        date = date_i[0]
    else:
        date = '{}.{}.{}'.format(date_i[0],date_i[1],date_i[2])
    information.update({'_id':i['_id']},{'$set':{'date':date}})

f:\setup\python3.5_win7\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [4]:
for i in information.find(): # 将地区为None的值改为不明
    if i['local']:
        local = i['local']
    elif i['local'] == None:
        local = ['不明']
    information.update({'_id':i['_id']},{'$set':{'local':local}})
   

f:\setup\python3.5_win7\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [29]:
def date_gen(date1, date2, date_step): # 日期生成函数
    start_date = date(int(date1.split('.')[0]),int(date1.split('.')[1]),int(date1.split('.')[2]))
    end_date = date(int(date2.split('.')[0]),int(date2.split('.')[1]),int(date2.split('.')[2]))
    step = timedelta(days = date_step)
    while start_date <= (end_date + timedelta(days = date_step)):   
        yield start_date.strftime("%Y.%m.%d")
        start_date = start_date + step
        

In [46]:
def get_data_gen(date1,date2,date_step,areas):   # 数据生成器
    for area in areas:
        d = []
        for date in date_gen(date1, date2, date_step):
            time_list = list(information.find({'local':area,'date':date}))
            d.append(len(time_list))
        data = {
            'name':area,
            'data':d,
            'type':'line'
        }
        yield data

In [50]:
# 固定的格式
options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': '科技公司融资情况'},
    'subtitle': {'text': '可视化统计图表'},
    'xAxis'   : {'categories': [i for i in date_gen('2016.11.17','2016.12.05',1)]},
    'yAxis'   : {'title': {'text': '数量'}}
    }
serise = [i for i in get_data_gen('2016.11.17','2016.12.05',1,['北京','上海','山东'])]

charts.plot(serise, show = 'inline', options = options)